In [ ]:
from os import path
import re

fullSub = re.compile(r'<(note|fw|num).+?/(note|fw|num)>')
wordSub = re.compile(r'</*w>|<w .*?>')
elemSub = re.compile(r'</*(lb|pb|pc|cb|hi|app|supplied|unclear|seg).*?>')
gabSub  = re.compile(r'<gap.*/>')
partSub = re.compile(r'<ab.*?part.*?>')
# puntSub = re.compile()

def mss_clean(input_file, output_path):
    outDict = {}
    bodyStart = False
    listWit = False
    metaData = ''
    witnesses = ''
    rawData = ''
#     out = open(output_file, 'w+')
    with open(input_file) as infile:
        filename = path.splitext(input_file)[0].split('/')[-1]
        
        for line in infile.readlines():
            if bodyStart == False:
                if '<body>' in line:
                    witnesses = [s.split('"')[1] for s in witnesses.split('><') if s.startswith('witness')]
                    for wit in witnesses:
                        outDict[f'{filename}-{wit}'] = metaData + '<body>'
                    bodyStart = True
                    continue
                if '<listWit>' in line:
                    listWit = True
                    witnesses += line.strip()
                if listWit == True:
                    witnesses += line.strip()
                if '</listWit>' in line:
                    witnesses += line.strip()
                    listWit = False
                metaData += line.strip()
            else:    
                line = re.sub(fullSub, '', line)
                line = re.sub(elemSub, '', line)
                line = re.sub(gabSub, ' {gap} ', line)
                line = re.sub(partSub, '', line)
                if line.strip().endswith('</w>'):
                    line = line.strip() + ' \n'
                line = re.sub(wordSub, '', line)
                if line.strip().endswith('>'):
                    line = line.strip() + '\n'
                line = line.replace('</ab>', '').replace('\n', '')
                rawData += line
            
        for wit in witnesses:
            out = open(f'{output_path}/{filename}-{wit}.xml', 'w+')
            outData = re.sub(rf'<rdg .*(?!{wit})+.*?</rdg>', '', rawData)
            out.write(outData)
            out.write(line.replace('</ab>', '').replace('\n', ''))
            out.close()

    
mss_clean(path.expanduser(path.join('~/github/pthu/sources/manuscripts/test/20004.xml')), path.expanduser(path.join('~/github/pthu/sources/manuscripts/test/out')))

In [ ]:
string = '<listWit><witness xml:id="firsthand"/><witness xml:id="corrector"/></listWit></sourceDesc>'
dictio = tuple(corr.split(' ')[1].strip for corr in [s for s in string.split('><') if s.startswith('witness')])
print(string.split('><'))
print([s.split('"')[1] for s in string.split('><') if s.startswith('witness')])

In [ ]:
import re

rawData = '<rdg type="orig" hand="firsthand"/><rdg type="corr" hand="corrector2"><w>γαρ</w></rdg><rdg type="corr" hand="correctorC"><w>γαρ</w></rdg>'
wit = 'firsthand'
# print(rawData)
# re.sub(r'<rdg .*?hand="(?!'+wit+')"((/>)|(.*?</rdg>))', '', rawData)
# re.sub(r'<rdg .*?hand=.*(?!'+wit+').*/>', '', rawData) #|<rdg .*?hand=.*(?!'+wit+').*</rdg>', '', rawData)

print(re.sub(r'<rdg.*?hand="((?<!'+wit+').)+".+?</rdg>', '', rawData))


# print(re.sub(r'<rdg.*?hand="(?<!'+wit+').*?"/>', '', rawData)) #|<rdg .*?hand=.*(?!'+wit+').*</rdg>', '', rawData)